In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

import numpy as np

from hdna import *

In [34]:
M = Model(standard=True)
M.setgeometry(theta=90, phi=180)
A = Strand(M, 'ACACACACACAC')
B = A.complementary()
K = Kinetics(M, A, B)

In [39]:
N = Kinetwork(M, A, B)    
N.save_graph('.')

((((((((((..+)))))))))).. 4.984e+12 0.000e+00 -15.596
((((((((....+)))))))).... 2.886e+11 0.000e+00 -12.209
((((((......+))))))...... 1.671e+10 0.000e+00 -8.823
((((........+))))........ 9.677e+08 0.000e+00 -5.436
((..........+)).......... 5.603e+07 0.000e+00 -2.049
..........((+..........)) 5.603e+07 0.000e+00 -2.049
........((((+........)))) 9.677e+08 0.000e+00 -5.436
......((((((+......)))))) 1.671e+10 0.000e+00 -8.823
....((((((((+....)))))))) 2.886e+11 0.000e+00 -12.209
..((((((((((+..)))))))))) 4.984e+12 0.000e+00 -15.596
2 4
((((((((((..+)))))))))).. ((((((((....+)))))))).... inchworming 2
2 6
((((((((((..+)))))))))).. ((((((......+))))))...... inchworming 4
2 8
((((((((((..+)))))))))).. ((((........+))))........ inchworming 6
2 10
((((((((((..+)))))))))).. ((..........+)).......... inchworming 8
2 -10
((((((((((..+)))))))))).. ..........((+..........)) pseudoknotting
2 -8
((((((((((..+)))))))))).. ........((((+........)))) pseudoknotting
2 -6
((((((((((..+)))))))))).. ......(((

In [30]:
N.connect_slidings()

((((((((..+)))))))).. 2.886e+11 0.000e+00 -12.209
((((((....+)))))).... 1.671e+10 0.000e+00 -8.823
((((......+))))...... 9.677e+08 0.000e+00 -5.436
((........+))........ 5.603e+07 0.000e+00 -2.049
........((+........)) 5.603e+07 0.000e+00 -2.049
......((((+......)))) 9.677e+08 0.000e+00 -5.436
....((((((+....)))))) 1.671e+10 0.000e+00 -8.823
..((((((((+..)))))))) 2.886e+11 0.000e+00 -12.209
2 4
((((((((..+)))))))).. ((((((....+)))))).... inchworming 2
2 6
((((((((..+)))))))).. ((((......+))))...... inchworming 4
2 8
((((((((..+)))))))).. ((........+))........ inchworming 6
2 -8
((((((((..+)))))))).. ........((+........)) pseudoknotting
2 -6
((((((((..+)))))))).. ......((((+......)))) pseudoknotting
2 -4
((((((((..+)))))))).. ....((((((+....)))))) pseudoknotting
2 -2
((((((((..+)))))))).. ..((((((((+..)))))))) pseudoknotting
4 2
((((((....+)))))).... ((((((((..+)))))))).. inchworming 2
4 6
((((((....+)))))).... ((((......+))))...... inchworming 2
4 8
((((((....+)))))).... ((........+)).

In [6]:
nodz = list(N.filternodes('state', lambda x: x == 'on_nucleation', N.DG).nodes())

objs = [N.DG.nodes[node]['obj'] for node in nodz]

In [7]:
for ob in objs:
    print(ob.structure)

(.........+.........)
.(........+........).
..(.......+.......)..
...(......+......)...
....(.....+.....)....
.....(....+....).....
......(...+...)......
.......(..+..).......
........(.+.)........
.........(+).........


In [8]:
def get_ix(string, char):
    indices = []
    for i, e in enumerate(list(string)):
        if e == char:
            indices.append(i)
    return indices 

In [9]:
on = 0
print(objs[on].structure)
l = objs[on].structure.split('+')[0]
r = objs[on].structure.split('+')[1][::-1]
ixl = get_ix(l,'(')[0]
ixr = get_ix(r,')')[0]

print(objs[on].s1.sdist[ixl])
objs[on].s1.sdist[ixl] == objs[on].s1.sdist[ixr]


(.........+.........)
0


True

In [10]:
np.ceil(2*SXGEO.PERSISTENCE/SXGEO.MONODIST)

7.0

In [11]:
print(f'{K.ksphere_sano():.3e}')

-6.350e+14


In [12]:
bottomheignt = 2*SXGEO.PERSISTENCE
topgyradius  = K.gyradiuses(N=4)
kc = K.ksphere_sano(a=topgyradius, badd=(bottomheignt))
print(f'{kc:.3e}')
kc = K.ksphere_sano(SXGEO.CY_RADIUS)
print(f'{kc:.3e}')
print()
print(SXGEO.CY_RADIUS)
print(K.gr1)

-5.507e+14
-5.218e+14

1e-07
2.1376996982738245e-07
